Autoencoder.
So that only one files needs to be submitted

In [ ]:
!pip install astropy
!pip install lightkurve
!pip install transitleastsquares

     |████████████████████████████████| 522kB 6.2MB/s 
     |████████████████████████████████| 6.5MB 25.9MB/s 
     |████████████████████████████████| 25.9MB 1.7MB/s 
     |████████████████████████████████| 256kB 40.1MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 2.6MB 49.3MB/s 
  Created wheel for astroquery: filename=astroquery-0.4.1-cp36-none-any.whl size=3831873 sha256=463873b2a4b67ffb8780e3708887ac8a12b70dbc90ad3ddb95006cc18ab1940c
  Stored in directory: /root/.cache/pip/wheels/88/f8/b7/a254cd96e808f708bc0b7d755a8e095c56fbbe94099d7b464f
  Created wheel for fbpca: filename=fbpca-1.0-cp36-none-any.whl size=11377 sha256=ae1ef6a26dce4441af7802d16f6b3cbb75483255b61ebcbee4e4a90b5f7aca53
  Stored in directory: /root/.cache/pip/wheels/53/a2/dd/9b66cf53dbc58cec1e613d216689e5fa946d3e7805c30f60dc
  Created wheel for oktopus: filename=oktopus-0.1.2-cp36-none-any.whl size=12779 sha256=d7c938395231b93d7b17928dfc32303cb65c3a317df014f4e39bc2878

In [ ]:
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
# This makes sure that we are using Tensorflow 2
try:
  # Note: %tensorflow_version only exists in Google Colab
  %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np

print(tf.__version__)
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

2.4.0
GPU Available:  []


In [ ]:
"""
This function adds no noise, we use this as a way of comparing the 
autoencoder when there's no input noise
"""
def no_noise(x):
  x = tf.dtypes.cast(x, tf.float32)
  return x

"""
This function should add a random uniform tensor of the shape of x between 
-0.3 to 0.3 to x. 

It should then "clip" x to between 0 and 1 (hint: check out tf.clip_by_value)
"""
def random_noise(x):
  x = tf.dtypes.cast(x, tf.float32)
  x = x + tf.random.uniform(tf.shape(x), -0.3, 0.3)
  
  noised = tf.clip_by_value(x, 0, 1)
  return noised

"""
This function should multiply a random uniform tensor of the shape of x between 
0 to 2.0 to x. 

It should then "clip" x to between 0 and 1 (hint: check out tf.clip_by_value)
"""
def random_scale(x):
  x = tf.dtypes.cast(x, tf.float32)
  x = x * tf.random.uniform(x.shape, minval=0, maxval= 2.0)

  noised = tf.clip_by_value(x, 0 ,1)
  return noised

def no_noise(x):
  """add no noise to the data"""
  return x
  
# some "unit tests"
x = [[0.3,0.1],[0.2,0]]
y = [[0.1,1],[0,0.33]]
result_1 = random_noise(x)
result_2 = random_scale(x)
result_3 = random_noise(y)
result_4 = random_scale(y)

for res in [result_1, result_2, result_3, result_4]:
  assert(res.shape == (2,2))
  assert(res.dtype == tf.float32)
  assert(np.max(np.array(res)) <= 1.0)
  assert(np.min(np.array(res)) >= 0.0)

print("Noise functions look good!")

Noise functions look good!


In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self):
       super(Encoder, self).__init__()
       self.encoder_conv_1 = tf.keras.layers.Conv1D(10, 50, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), padding='SAME', bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), strides=2, activation=tf.keras.layers.LeakyReLU(alpha=0.2))
       #self.encoder_conv_2 = tf.keras.layers.Conv1D(10, 3, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), padding='SAME', bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), strides=2, activation=tf.keras.layers.LeakyReLU(alpha=0.2))
       #self.encoder_conv_3 = tf.keras.layers.Conv1D(10, 3, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), padding='SAME', bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), strides=2, activation=tf.keras.layers.LeakyReLU(alpha=0.2))  
    
    @tf.function
    def call(self, images):

      return self.encoder_conv_1(images) # self.encoder_conv_3(self.encoder_conv_2(self.encoder_conv_1(images))) 

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self):
        super(Decoder, self).__init__()

        self.decoder_deconv_1 = tf.keras.layers.Conv1DTranspose(1, 50, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), padding='SAME', strides=2, bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), activation=tf.keras.layers.LeakyReLU(alpha=0.2))
        #self.decoder_deconv_2 = tf.keras.layers.Conv1DTranspose(10, 3, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), padding='SAME', strides=2, bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), activation=tf.keras.layers.LeakyReLU(alpha=0.2))
        #self.decoder_deconv_3 = tf.keras.layers.Conv1DTranspose(1, 3, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), padding='SAME', strides=2, bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1), activation=tf.keras.layers.LeakyReLU(alpha=0.2))
        
    
        
    
    @tf.function
    def call(self, encoder_output):
        
        return self.decoder_deconv_1(encoder_output) #self.decoder_deconv_3(self.decoder_deconv_2(self.decoder_deconv_1(encoder_output)))  

In [ ]:
class AutoEncoder(tf.keras.Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
    
    @tf.function
    def call(self, images):
       return self.decoder.call(self.encoder.call(images))
    
    @tf.function
    def loss_function(self, encoded, originals):
      encoded = tf.dtypes.cast(encoded, tf.float32)
      originals = tf.dtypes.cast(originals, tf.float32)
      return tf.keras.backend.sum(tf.keras.backend.square(encoded- originals))

In [ ]:
def train(model, optimizer, images, noise_function):
  corrupted = noise_function(images)
  
  with tf.GradientTape() as tape:
    encoded = model.call(corrupted)
    loss = model.loss_function(encoded, corrupted)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
def total_loss(model, images, noise_function):
  sum_loss = 0
  corrupted = noise_function(images)
  predictions = model(corrupted)
  sum_loss += model.loss_function(predictions, corrupted).numpy()
  return sum_loss

Application of autoencoder


# Downloading data from TESS


#fit a physics model
1. first use a simple boxed least squares model to find the transit 
period (fast) 
2. then use a more advanced model to get other parameters for an estimation of how well the lightcurve fits a physics model
3. output some scores of goodness of fit and best parameters

In [ ]:
from transitleastsquares import transitleastsquares

def fit_model(lc):
  pg = lc.to_periodogram(method='bls',duration=0.2, minimum_period=1.0)
  pg.plot();
  lc.fold(pg.period_at_max_power).scatter();
  print(pg.period_at_max_power.value)

  model = transitleastsquares(lc.time, lc.flux)
  results = model.power(period_min=pg.period_at_max_power.value*0.9, period_max=pg.period_at_max_power.value*1.1)
  results.CDPP = lc.estimate_cdpp()
  return results

def plot_results(results):
  plt.figure()
  ax = plt.gca()
  ax.axvline(results.period, alpha=0.4, lw=3)
  plt.xlim(np.min(results.periods), np.max(results.periods))
  for n in range(2, 10):
      ax.axvline(n*results.period, alpha=0.4, lw=1, linestyle="dashed")
      ax.axvline(results.period / n, alpha=0.4, lw=1, linestyle="dashed")
  plt.ylabel(r'SDE')
  plt.xlabel('Period (days)')
  plt.plot(results.periods, results.power, color='black', lw=0.5)
  plt.xlim(0, max(results.periods));

def print_results(results):
  print('Period', format(results.period, '.5f'), 'd')
  #print(len(results.transit_times), 'transit times in time series:', \
          #['{0:0.5f}'.format(i) for i in results.transit_times])
  print('Transit depth', format(results.depth, '.5f'))
  print('Transit duration (days)', format(results.duration, '.5f'))
  print('SNR', results.snr)
  print('chi2', results.chi2_min)
  print('CDPP', results.CDPP)

def plot_result_fit(results):
  plt.figure()
  plt.plot(results.model_folded_phase, results.model_folded_model,color='red')
  plt.scatter(results.folded_phase, results.folded_y, color='blue', s=10, alpha=0.5, zorder=2)
  plt.xlabel('Phase')
  plt.ylabel('Relative flux');

def fit_and_report(lc):
  results = fit_model(lc)
  print_results(results)
  plot_results(results)
  plot_result_fit(results)

#compare results to lightcurves generated from Tess

Get dataset and make them all 1D

In [ ]:
raw_data = lk.search_lightcurvefile('HAT-P-11', sector=14, cadence='short').download()
image_lc = raw_data.get_lightcurve('SAP_FLUX')

# flatten (detrend) the lightcurve to show relative change using window of 1 day
samples_per_day = 1 + 60 * 24 // 2
flat_image_lc = image_lc.flatten(samples_per_day);


flux = flat_image_lc.flux
flux = tf.convert_to_tensor(flux)
flux = tf.reshape(flux, (1, 18424, 1))

Run encoder model on dataset

In [ ]:
n_epochs = 500
model = AutoEncoder()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
flux = tf.dtypes.cast(flux, tf.float32)

for epoch in range(n_epochs):
    train(model, optimizer, flux, no_noise)
    sum_loss = total_loss(model, flux, no_noise)

In [ ]:
flux_output = model(flux).numpy()[0]

new_flux = []
for i in range(len(flux_output)):
  new_flux.append(flux_output[i][0])
new_flux = np.array(new_flux)

flat_image_lc.flux = new_flux
flat_image_lc.plot()

fit_and_report(flat_image_lc)

query_solar_system_objects(self[, …]) Returns a list of asteroids or comets which affected the light curve.

remove_outliers(self[, sigma, sigma_lower, …]) Removes outlier data points using sigma-clipping.

search_result = search_targetpixelfile('Kepler-10')  